In [200]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [201]:
import warnings
warnings.filterwarnings("ignore")

In [202]:
# os.chdir("Desktop")

In [203]:
df= pd.read_csv("wbc.csv")

In [204]:
df.shape

(569, 33)

In [205]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
id                         569 non-null int64
diagnosis                  569 non-null object
radius_mean                569 non-null float64
texture_mean               569 non-null float64
perimeter_mean             569 non-null float64
area_mean                  569 non-null float64
smoothness_mean            569 non-null float64
compactness_mean           569 non-null float64
concavity_mean             569 non-null float64
concave points_mean        569 non-null float64
symmetry_mean              569 non-null float64
fractal_dimension_mean     569 non-null float64
radius_se                  569 non-null float64
texture_se                 569 non-null float64
perimeter_se               569 non-null float64
area_se                    569 non-null float64
smoothness_se              569 non-null float64
compactness_se             569 non-null float64
concavity_se               569 non

In [206]:
#Remove redundant or blank columns
df.drop(["Unnamed: 32","id"],1,inplace=True)

In [207]:
#Check for missing values
df.isnull().sum()

diagnosis                  0
radius_mean                0
texture_mean               0
perimeter_mean             0
area_mean                  0
smoothness_mean            0
compactness_mean           0
concavity_mean             0
concave points_mean        0
symmetry_mean              0
fractal_dimension_mean     0
radius_se                  0
texture_se                 0
perimeter_se               0
area_se                    0
smoothness_se              0
compactness_se             0
concavity_se               0
concave points_se          0
symmetry_se                0
fractal_dimension_se       0
radius_worst               0
texture_worst              0
perimeter_worst            0
area_worst                 0
smoothness_worst           0
compactness_worst          0
concavity_worst            0
concave points_worst       0
symmetry_worst             0
fractal_dimension_worst    0
dtype: int64

In [208]:
#Evaluate the target variable for class imbalance
df.diagnosis.value_counts(normalize=True)

B    0.627417
M    0.372583
Name: diagnosis, dtype: float64

In [209]:
#Slightly imbalanced data set- we could use SMOTE to oversample the minority class- I will do this post EDA

In [210]:
#EDA- lets get some basic insights from the data

In [211]:
#Lets split the data into the benign category vs. malignant category to see the differences
ben= df.loc[df["diagnosis"]=="B"]
mal= df.loc[df["diagnosis"]=="M"]

In [212]:
#Lets get the means of all the different variables for the two classes
mean_list_ben= ben.describe().mean()

In [213]:
mean_list_ben.index[0]

'radius_mean'

In [214]:
mean_list_mal= mal.describe().mean()

In [215]:
mean_list_length= np.arange(len(mean_list_ben))

In [216]:
malignant_higher_mean= []
benign_higher_mean= []
for x in mean_list_length:
    if mean_list_ben[x]> mean_list_mal[x]:
        benign_higher_mean.append(mean_list_ben.index[x])
    else:
        malignant_higher_mean.append(mean_list_ben.index[x])

In [217]:
len(benign_higher_mean)

25

In [218]:
len(malignant_higher_mean)

5

In [219]:
#As we can see, most of the indicators for the benign category have a higher mean than the malignant category
#Only 5 indicators have a higher mean in the malignant category
#Let us take only these 5 indicators in our base model and see what accuracy we get

In [220]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [221]:
#Preprocessing the target variable into a continuous output
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
le.fit(df.diagnosis)
y= le.fit_transform(df.diagnosis)

In [222]:
#Standardizing all the feature variables 
X_df= df.drop(["diagnosis"],1)
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
scaler.fit(X_df)
X= scaler.transform(X_df)

In [223]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split as tts

In [224]:
sm = SMOTE(random_state=42)

In [225]:
X_train,X_test,y_train,y_test = tts(X,y,test_size = 0.2,random_state = 42)

In [226]:
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())

In [227]:
from sklearn.linear_model import LogisticRegression

In [228]:
logreg= LogisticRegression()

In [229]:
logreg.fit(X_train_res,y_train_res)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [230]:
y_pred_logreg= logreg.predict(X_test)

In [231]:
from sklearn.metrics import accuracy_score

In [232]:
accuracy_score(y_test,y_pred_logreg)

0.9824561403508771

In [233]:
from sklearn.metrics import classification_report

In [234]:
classification_report(y_test,y_pred_logreg)

'              precision    recall  f1-score   support\n\n           0       0.99      0.99      0.99        71\n           1       0.98      0.98      0.98        43\n\n   micro avg       0.98      0.98      0.98       114\n   macro avg       0.98      0.98      0.98       114\nweighted avg       0.98      0.98      0.98       114\n'

In [235]:
from sklearn.metrics import roc_auc_score

In [236]:
roc_auc_score(y_test,y_pred_logreg)

0.981329839502129

In [237]:
#Cross validation to reduce overfitting
from sklearn.model_selection import GridSearchCV

In [238]:
# Create regularization penalty space
penalty = ['l1', 'l2']

# Create regularization hyperparameter space
C = np.logspace(0, 4, 10)

# Create hyperparameter options
hyperparameters = dict(C=C, penalty=penalty)

In [239]:
clf = GridSearchCV(logreg, hyperparameters, cv=5, verbose=0)

In [240]:
clf.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([1.00000e+00, 2.78256e+00, 7.74264e+00, 2.15443e+01, 5.99484e+01,
       1.66810e+02, 4.64159e+02, 1.29155e+03, 3.59381e+03, 1.00000e+04]), 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [241]:
y_pred_gscv= clf.predict(X_test)

In [242]:
accuracy_score(y_test,y_pred_gscv)

0.9736842105263158

In [243]:
roc_auc_score(y_test,y_pred_gscv)

0.9742875859810023

In [244]:
#Grid search CV seems to have only slightly reduced the extent of overfitting

In [245]:
#Lets use another algorithm that is less prone to high variance/overfitting: Naive Bayes
from sklearn.naive_bayes import GaussianNB

In [246]:
nb= GaussianNB()

In [247]:
nb.fit(X_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [248]:
y_pred_nb= nb.predict(X_test)

In [249]:
accuracy_score(y_test,y_pred_nb)

0.9649122807017544

In [250]:
roc_auc_score(y_test,y_pred_nb)

0.9580740255486406

In [251]:
#Lets try using a bagging (ensemble) method in order to reduce overfitting/variance- random forest 

In [252]:
from sklearn.ensemble import RandomForestClassifier

In [253]:
rfc= RandomForestClassifier(n_estimators= 300)

In [254]:
rfc.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [255]:
y_pred_rfc= rfc.predict(X_test)

In [256]:
accuracy_score(y_test,y_pred_rfc)

0.9649122807017544

In [257]:
roc_auc_score(y_test,y_pred_rfc)

0.9580740255486406

In [258]:
importances_rfc = pd.Series(rfc.feature_importances_,
                          index = X_df.columns)
sorted_importances = importances_rfc.sort_values(ascending=False)
sorted_importances.head(30)

concave points_worst       0.134819
area_worst                 0.118123
concave points_mean        0.109918
radius_worst               0.105181
perimeter_worst            0.092799
concavity_mean             0.054003
perimeter_mean             0.052669
concavity_worst            0.041604
area_mean                  0.040264
area_se                    0.035846
radius_mean                0.034627
texture_worst              0.020861
radius_se                  0.016525
symmetry_worst             0.015998
compactness_mean           0.015338
texture_mean               0.015166
compactness_worst          0.014285
smoothness_worst           0.012774
perimeter_se               0.009935
smoothness_mean            0.007860
concavity_se               0.007009
texture_se                 0.005658
fractal_dimension_se       0.005507
fractal_dimension_worst    0.005504
smoothness_se              0.005364
symmetry_se                0.004887
symmetry_mean              0.004653
compactness_se             0

In [259]:
#Lets try another ensemble method to see if we can further reduce overfitting- voting classifier
from sklearn.ensemble import VotingClassifier

In [260]:
vc = VotingClassifier(estimators=[('rfc', rfc), ('nb', nb), ('logreg', logreg)])

In [261]:
vc.fit(X_train, y_train)

VotingClassifier(estimators=[('rfc', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_w...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [262]:
y_pred_vc = vc.predict(X_test)

In [263]:
accuracy_score(y_test,y_pred_vc)

0.9649122807017544

In [264]:
roc_auc_score(y_test,y_pred_vc)

0.9580740255486406

In [265]:
classification_report(y_test,y_pred_vc)

'              precision    recall  f1-score   support\n\n           0       0.96      0.99      0.97        71\n           1       0.98      0.93      0.95        43\n\n   micro avg       0.96      0.96      0.96       114\n   macro avg       0.97      0.96      0.96       114\nweighted avg       0.97      0.96      0.96       114\n'

In [266]:
#It seems that all the models attempted show a high degree of overfitting - with all scoring abovr 95%
#I would select the voting classifier as my final model for this dataset